# Tutorial
based on <a href='https://www.youtube.com/watch?v=jTDnGox0c9Y'>link</a>

- define the objective (minimisation)
- define the contraints in terms of sums equalities/inequalities
- build the objective piece by piece

In [21]:
from dimod import BinaryQuadraticModel

In [22]:
bqm = BinaryQuadraticModel('BINARY') #BINARY means I'm using 0/1 opposite to -1/+1

In [23]:
pumps = [0,1,2,3]
time = [0,1]
costs = [[36,27],
       [56,65],
       [48,36],
       [52,16]]
flow = [2,7,3,8]
demand = 20

#Build a variable for each pump
x = [[f'P{p}_AM',f'P{p}_PM'] for p in pumps]

#### build the objective piece by piece

In [24]:
for p in pumps:
    for t in time:
        bqm.add_variable(x[p][t],costs[p][t])

#### constraint 1 (once a day)
The bias is 1. There is no coefficient. There is also an implicit upper bound (only 2 time slots)

In [25]:
# c1 = list of binary variables with coefficients 1
for p in pumps:
    c1 = [(x[p][t],1) for t in time] # 1 is the biasc
    print(c1)
    bqm.add_linear_inequality_constraint(
        c1,
        lb =1, #lower bound
        ub = len(time), #upper bound
        lagrange_multiplier = 13, #penalty coefficient ?
        label = 'c1_pump_'+str(p)
    )
    

[('P0_AM', 1), ('P0_PM', 1)]
[('P1_AM', 1), ('P1_PM', 1)]
[('P2_AM', 1), ('P2_PM', 1)]
[('P3_AM', 1), ('P3_PM', 1)]


#### constraint 2 (max 3 pumps at a time)
Move the 3 so it becomes sum x -3 >= 0. The bias is 1, upper bound is 3. There is a constant of -3

In [26]:
for t in time:
    c2 = [(x[p][t], 1) for p in pumps] #bias = 1
    bqm.add_linear_inequality_constraint(
        c2,
        constant= -3,
        lagrange_multiplier = 1,
        label = 'c2_time_'+str(t)
        )

#### constraint 3 (satisfy daily demand)
Move the demand so it becomes sum x -demand = 0. The bias is the flow. There is a constant of -demand

In [27]:
c3 = [(x[p][t],flow[p]) for t in time for p in pumps] #flow is the bias
bqm.add_linear_equality_constraint(
    c3,
    constant= - demand,
    lagrange_multiplier = 28
    )

# Run

In [30]:
from dwave.system import DWaveSampler, EmbeddingComposite

In [31]:
sampler = EmbeddingComposite(DWaveSampler()) #ocean decides where to fit the logical problem on the physical chip structure
sampleset = sampler.sample(bqm, num_reads = 10)

sample = sampleset.first.sample

In [32]:
sample

{'P0_AM': 0,
 'P0_PM': 1,
 'P1_AM': 0,
 'P1_PM': 0,
 'P2_AM': 0,
 'P2_PM': 1,
 'P3_AM': 1,
 'P3_PM': 1,
 'slack_c1_pump_0_0': 1,
 'slack_c1_pump_1_0': 1,
 'slack_c1_pump_2_0': 0,
 'slack_c1_pump_3_0': 0,
 'slack_c2_time_0_0': 0,
 'slack_c2_time_0_1': 0,
 'slack_c2_time_1_0': 1,
 'slack_c2_time_1_1': 0}